#### Do not run this cell unless you want to download the RT-GTFS from DTC website

In [2]:
# import requests
# import csv
# import gtfs_realtime_pb2
# import time

# PB_URL = "https://otd.delhi.gov.in/api/realtime/VehiclePositions.pb?key=qQdrLdHL9z5X2IEPMv1JNx524TutmrQV"

# positions = str

# cols = ["entity_id","trip","route","start_time","start_date","vehicle_id","lat","lon","veh_speed","veh_timestamp"]
# iteration = 1
# while True: # Code will run forever without this while statement
#     response = requests.get(PB_URL)
#     response.raise_for_status()

#     feed_message = gtfs_realtime_pb2.FeedMessage()

#     feed_message.ParseFromString(response.content)

#     with open('output/dtc-rt/dtc-rt-gtfs-output.csv','a',newline='') as csv_file:
#         data = []
#         writer = csv.writer(csv_file)

#         trips,vehicle,position,timestamp = None,None,None,None

#         for entity in feed_message.entity:
#             entity_id = entity.id
#             trip = entity.vehicle.trip.trip_id
#             route = entity.vehicle.trip.route_id
#             start_time = entity.vehicle.trip.start_time
#             start_date = entity.vehicle.trip.start_date

#             vehicle_id = entity.vehicle.vehicle.id

#             lat = entity.vehicle.position.latitude
#             lon = entity.vehicle.position.longitude
#             veh_speed = entity.vehicle.position.speed

#             veh_timestamp = entity.vehicle.timestamp
#             # print(entity)
#             data.append([entity_id,trip,route,start_time,start_date,vehicle_id,lat,lon,veh_speed,veh_timestamp])
#         print(f"Iteration No: {iteration}")
#         iteration += 1
#         print(len(data))
#         writer.writerows(data)
#     time.sleep(3)

In [3]:
# my_map = folium.Map(None)
# coords = []
# route = None
# end_time,start_time,veh_id = None,None, None


# grp_fg = folium.FeatureGroup().add_to(my_map)
# # print(len(grouped_df))
# for key,my_df in grouped_df:
#     route = key[0]
#     vehicle = key[1]
#     if route >= 5000:
#         start_time = my_df.iloc[0]['start_datetime']
#         end_time = my_df.iloc[len(my_df)-1]['start_datetime']
        
#         c = [[item['lat'],item['lon']] for _, item in my_df.iterrows()]
        
#         coords.append(c)
        
#         folium.PolyLine(c,tooltip = f"Vehicle ID: {veh_id}<br>Route: {route}",color=rgb_to_hex()).add_to(grp_fg)

In [4]:
import pandas as pd
import folium
import random
from shapely.geometry import Point
import math
import pyproj

In [5]:
df = pd.read_csv('output/dtc-rt/dtc-rt-gtfs-output.csv',parse_dates=['start_time','start_date'])

cols = ["entity_id","trip","route","start_time","start_date","vehicle_id","lat","lon","veh_speed","veh_timestamp"]

In [6]:
# Filter only DTC Routes
df = df.loc[df['route']>=5000].reset_index()
df['start_datetime'] = pd.to_datetime(pd.to_datetime(df['start_date']).dt.strftime('%Y-%m-%d') + ' ' + pd.to_datetime(df['start_time']).dt.strftime('%H:%M:%S'))
df['veh_timestamp'] = pd.to_datetime(df['veh_timestamp'])

# Group by vehicle_id and sort by start_time
df = df.sort_values(by=['route','vehicle_id','start_datetime'],ascending=True)

In [7]:
def rgb_to_hex():
    color = random.choices(range(150),k=3)

    r = color[0]
    g = color[1]
    b = color[2]
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

In [8]:
df['lat'] = df['lat'].round(4)
df['lon'] = df['lon'].round(4)

In [9]:
marker_collection = df.groupby(['lat','lon']).size().reset_index()

In [ ]:
my_map = folium.Map(None)
grp_fg = folium.FeatureGroup().add_to(my_map)

collection = marker_collection.groupby(by=0)

for key,mk in collection:
    mk.reset_index()
    r = int(1.5)
    # print(mk['lat'])
    colors = rgb_to_hex()
    for _,item in mk.iterrows():
        # print(item['lat'])
        folium.CircleMarker([float(item['lat']),item['lon']],radius = r,color = colors,tooltip = f"{key}").add_to(grp_fg)
    

In [ ]:
my_map.fit_bounds(grp_fg.get_bounds())
my_map #.save('routes.html')